In [1]:
import time
from datetime import datetime as dt
from biscoint_api_python import Biscoint
from playsound import playsound
from utils import percent, btcToTrade
from configs import logging
from configs import API_KEY,API_SECRET,AMOUNT,MIN_PERCENT_REQUIRED,BASE_URL,BRL_AMOUNT_TRADE

In [2]:
bsc = Biscoint(API_KEY, API_SECRET)
endpoints_meta = bsc.get_meta()
rate_limit_offer = endpoints_meta['endpoints']['offer']['post']['rateLimit']
sleep_time_offers = ((rate_limit_offer["windowMs"] / rate_limit_offer["maxRequests"]) / 1000)*2
initial_balance = bsc.get_balance()
last_balance = initial_balance
ticker = bsc.get_ticker()
amount_btc_to_trade = btcToTrade(BRL_AMOUNT_TRADE,ticker['askQuoteAmountRef'],ticker['bidBaseAmountRef'])

In [14]:
#requests.get(f"{BASE_URL}/fees").json()
#bsc.get_fees()
cycle_count = 3

cycle_count % 2 == 0

False

In [17]:
cycle_count = 1
while True:
    start_time = dt.now()
    buy = bsc.get_offer(op='buy',amount=str(amount_btc_to_trade),isQuote=False)
    sell = bsc.get_offer(op='sell',amount=str(amount_btc_to_trade),isQuote=False)

    calculated_percent = percent(buy['efPrice'],sell['efPrice'])
    print(f"Percent is {calculated_percent}")

    if(calculated_percent >= MIN_PERCENT_REQUIRED):
        print(f"Arbitrage oportunity: buy:{buy['efPrice']}   sell:{sell['efPrice']}")
        playsound('beep.wav')
        #Execute orders
        executed_sell = bsc.confirm_offer(sell['offerId'])
        executed_buy = bsc.confirm_offer(buy['offerId'])
        last_balance = bsc.get_balance()
        break


    end_time = dt.now()
    seconds_elapsed = (end_time - start_time).total_seconds()

    #print(f"Took {seconds_elapsed} seconds")
    if cycle_count % 2 == 0:
        ticker = bsc.get_ticker()
        amount_btc_to_trade = btcToTrade(BRL_AMOUNT_TRADE,ticker['askQuoteAmountRef'],ticker['bidBaseAmountRef'])   

    cycle_count +=1
    time.sleep(sleep_time_offers)

Percent is -0.777
Percent is -0.741
Percent is -0.74
Percent is -0.74
Percent is -0.74
Percent is -0.74
Percent is -0.74
Percent is -0.668


KeyboardInterrupt: 

In [ ]:
#thread_buy = threading.Thread(target=offerFunc,args=('buy',AMOUNT,False,buy))
#thread_sell = threading.Thread(target=offerFunc,args=('sell',AMOUNT,False,sell))
#thread_buy.start()
#thread_sell.start()
#thread_buy.join()
#thread_sell.join()
